Kaggle Competition Link: https://www.kaggle.com/competitions/titanic/overview

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("train.csv")
X_output = pd.read_csv("test.csv")


In [38]:
data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [25]:
#Used to create submission file, since there is no "Survived" feature present
X_output.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Data Cleaning

In [100]:
def clean_data(input_data):
    #Things we don't want to include: strings, Null values
    
    dataframe = input_data.copy()
    
    #Sex: Binary Encoding
    dataframe['is_female'] = dataframe['Sex'].map({"female": 1, "male": 0})
    dataframe = dataframe.drop(columns = ["Sex"])

    #Age: Fill Null values (mean)
    mean_age = round(dataframe['Age'].mean(), 1)
    dataframe['Age'].fillna(mean_age, inplace=True)
    
    #Fare: Fill Null Values (mean)
    mean_fare = round(dataframe['Fare'].mean(), 1)
    dataframe['Fare'].fillna(mean_fare, inplace=True)

    #Ticket: Remove characters
    dataframe['Ticket'] = dataframe['Ticket'].str.replace(r'\D', '', regex=True)

    #Embarked: Label Encoding, fill Null Values (mode)
    dataframe['Embarked'] = dataframe['Embarked'].map({"C": 0, "Q": 1, "S": 2})
    embarked_mode = dataframe['Embarked'].mode()[0]
    dataframe['Embarked'].fillna(embarked_mode, inplace=True)

    #Drop difficult to encode columns (For your better model you'll want to use a more sophisticated data cleaning technique)
    dataframe = dataframe.drop(columns = ['Name', 'Cabin', 'Ticket'])
    
    return dataframe

    
data = pd.read_csv("train.csv")
data = clean_data(data)

dataframe.head(10)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,is_female
0,1,0,3,22.0,1,0,7.2500,2.0,0
1,2,1,1,38.0,1,0,71.2833,0.0,1
2,3,1,3,26.0,0,0,7.9250,2.0,1
3,4,1,1,35.0,1,0,53.1000,2.0,1
4,5,0,3,35.0,0,0,8.0500,2.0,0
5,6,0,3,29.7,0,0,8.4583,1.0,0
6,7,0,1,54.0,0,0,51.8625,2.0,0
7,8,0,3,2.0,3,1,21.0750,2.0,0
8,9,1,3,27.0,0,2,11.1333,2.0,1
9,10,1,2,14.0,1,0,30.0708,0.0,1


In [80]:
y = data["Survived"]
X = data.drop(columns = ["Survived"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [81]:
print(f'X_train Samples: {X_train.shape[0]}, X_train Features: {X_train.shape[1]};  y_train Samples: {y_train.shape[0]}')
print(f'X_test Samples: {X_test.shape[0]}, X_test Features: {X_test.shape[1]};  y_test Samples: {y_test.shape[0]}')

X_train Samples: 712, X_train Features: 8;  y_train Samples: 712
X_test Samples: 179, X_test Features: 8;  y_test Samples: 179


# Models

In [83]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

#Decision Tree
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

dt_y_pred = dt.predict(X_test)
dt_acc = accuracy_score(y_test, dt_y_pred)

print(f'Decision Tree Accuracy: {dt_acc}')

Decision Tree Accuracy: 0.7318435754189944


In [85]:
from sklearn.ensemble import RandomForestClassifier

#Random Forest: Ensemble method on Decision Tree
rf = RandomForestClassifier(n_estimators = 100, random_state = 42)

rf.fit(X_train, y_train)

rf_y_pred = rf.predict(X_test)
rf_acc = accuracy_score(y_test, rf_y_pred)

print(f'Random Forest Accuracy: {rf_acc}')

Random Forest Accuracy: 0.8156424581005587


# Generating an output file

In [101]:
X_output = pd.read_csv("test.csv")
X_output = clean_data(X_output)

X_output.head(10)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Embarked,is_female
0,892,3,34.5,0,0,7.8292,1,0
1,893,3,47.0,1,0,7.0000,2,1
2,894,2,62.0,0,0,9.6875,1,0
3,895,3,27.0,0,0,8.6625,2,0
4,896,3,22.0,1,1,12.2875,2,1
5,897,3,14.0,0,0,9.2250,2,0
6,898,3,30.0,0,0,7.6292,1,1
7,899,2,26.0,1,1,29.0000,2,0
8,900,3,18.0,0,0,7.2292,0,1
9,901,3,21.0,2,0,24.1500,2,0


In [103]:
#Create output
y_pred = rf.predict(X_output)

output_data = pd.DataFrame({'PassengerId': X_output['PassengerId'], 'y_pred': y_pred})
output_data.to_csv('output_file.csv', index=False)